<a href="https://colab.research.google.com/github/emisoft-designs/Data-Science-Practice-Projects/blob/main/Neural%20network%20Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **HOW TO CREATE A BACKPROPAGATION NEURAL NETWORK**

In [ ]:
from random import seed
from random import random
from math import exp
import pandas as pd
import numpy as np

### 1. Initialize Network

In [ ]:
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
  network = list()
  hidden_layers = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
  network.append(hidden_layers)
  output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
  network.append(output_layer)
  return network

In [ ]:
# test network
network = initialize_network(2, 1, 2)
for layer in network:
  print(layer)

[{'weights': [0.010990261875375884, 0.455707281052777, 0.383060273071289]}]
[{'weights': [0.1262972817749176, 0.738269143945797]}, {'weights': [0.008691397527751388, 0.8293968945082779]}]


## 2. Forward Propagate

#### 2.1 Neuron Activation

In [ ]:
# Calculate neuron activation for an input
# activation = sum(weight_i * input_i) + bias
def activate(weights, inputs):
  activation = weights[-1]
  for i in range(len(weights)-1):
    activation += weights[i] * inputs[i]
  return activation

### 2.2 Neuron Transfer

In [ ]:
# ReLU = max(0,x)
def transfer(activation):
  return 1.0/(1.0 + exp(-activation))

# 2.3 Forward Propagation

In [ ]:
# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

In [ ]:
# test forward propagation
network = [[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
		[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]]
row = [1, 0, None]
output = forward_propagate(network, row)
print(output)


[0.6629970129852887, 0.7253160725279748]


## **3. Back Propagate Error**

### **3.1 Transfer Derivative**

In [ ]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)

### **3.2 Error Backpropagation**

In [ ]:
# output neuron error = (output - expected) * transfer_derivative(output)
# hidden neuron error = (weight_k * error_j) * transfer_derivative(output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(neuron['output'] - expected[j])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [ ]:
# test backpropagation of error
network = [[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
		[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]}, 
   {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]]
expected = [0, 1]
backward_propagate_error(network, expected)
for layer in network:
	print(layer)

[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'delta': 0.0005348048046610517}]
[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': 0.14619064683582808}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': -0.0771723774346327}]


## **4. Train Network**

### 4.1 Update Weights

In [ ]:
# update weight = weight - learning_rate * error * input


# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] -= l_rate * neuron['delta']

### 4.2 Train Network

In [ ]:

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[int(row[-1])] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [ ]:
# Test training backprop algorithm
seed(1)
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.7, 30, n_outputs)
for layer in network:
	print(layer)

>epoch=0, lrate=0.700, error=6.205
>epoch=1, lrate=0.700, error=5.370
>epoch=2, lrate=0.700, error=5.090
>epoch=3, lrate=0.700, error=4.551
>epoch=4, lrate=0.700, error=4.078
>epoch=5, lrate=0.700, error=3.614
>epoch=6, lrate=0.700, error=3.187
>epoch=7, lrate=0.700, error=2.799
>epoch=8, lrate=0.700, error=2.452
>epoch=9, lrate=0.700, error=2.146
>epoch=10, lrate=0.700, error=1.882
>epoch=11, lrate=0.700, error=1.654
>epoch=12, lrate=0.700, error=1.459
>epoch=13, lrate=0.700, error=1.292
>epoch=14, lrate=0.700, error=1.150
>epoch=15, lrate=0.700, error=1.028
>epoch=16, lrate=0.700, error=0.923
>epoch=17, lrate=0.700, error=0.832
>epoch=18, lrate=0.700, error=0.754
>epoch=19, lrate=0.700, error=0.685
>epoch=20, lrate=0.700, error=0.624
>epoch=21, lrate=0.700, error=0.571
>epoch=22, lrate=0.700, error=0.524
>epoch=23, lrate=0.700, error=0.482
>epoch=24, lrate=0.700, error=0.445
>epoch=25, lrate=0.700, error=0.413
>epoch=26, lrate=0.700, error=0.383
>epoch=27, lrate=0.700, error=0.357
>e

## **Predict**

In [ ]:
# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

In [ ]:
# Test making predictions with the network
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
network = [[{'weights': [-1.482313569067226, 1.8308790073202204, 1.078381922048799]}, {'weights': [0.23244990332399884, 0.3621998343835864, 0.40289821191094327]}],
	[{'weights': [2.5001872433501404, 0.7887233511355132, -1.1026649757805829]}, {'weights': [-2.429350576245497, 0.8357651039198697, 1.0699217181280656]}]]
for row in dataset:
	prediction = predict(network, row)
	print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1


# **FULL ARCHITECTURE**

In [ ]:
# Backprop on the Seeds Dataset
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp
 
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset
 
# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())
 
# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup
 
# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	stats = [[min(column), max(column)] for column in zip(*dataset)]
	return stats

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
 
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores
 
 

# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation

# Transfer neuron activation
def transfer(activation):
	return 1.0 / (1.0 + exp(-activation))

# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					print(neuron)
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(neuron['output'] - expected[j])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] -= l_rate * neuron['delta']

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[int(row[-1])] = 1
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)

# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))
 
# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = initialize_network(n_inputs, n_hidden, n_outputs)
	train_network(network, train, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)


In [ ]:
def clean_string(x):
  num_list = x[0].replace("\t",',')
  num_list = num_list.split(',')
  float_list = [float(x) for x in num_list if x != '']
  return float_list

In [ ]:
# Test Backprop on Seeds dataset
seed(1)

# load and prepare data
filename = '/content/drive/MyDrive/Colab Notebooks/Development lab/seeds_dataset.csv'
dataset = load_csv(filename)

for i in range(len(dataset)):
  try:
	  dataset[i] = clean_string(dataset[i])
  except ValueError:
    print("line ->", i)

In [ ]:
minmax = dataset_minmax(dataset)
range(len(minmax))

range(0, 8)

In [ ]:
range(len(dataset[7]))

range(0, 8)

In [ ]:
dataset[7]

[14.11, 14.1, 0.8911, 5.42, 3.302, 2.7, 5.0, 1.0]

In [ ]:
 # normalize input variables
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

# set optimization parameters
n_folds = 5
l_rate = 0.3
n_epoch = 1
n_hidden = 5

# evaluate algorithm
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Streaming output truncated to the last 5000 lines.
{'weights': [0.6085709258038992, 0.35039493999854443, 0.5189410337069124, 0.04008847810032917, 0.26070612953530903, 0.234014318018005], 'output': 0.8259650790226697, 'delta': 0.11872980997729642}
{'weights': [0.1122737752486256, 0.09239395828446864, 0.7429583733132444, 0.916614253534063, 0.5980192146330156, 0.7852987254384954], 'output': 0.9357102332007124, 'delta': -0.0038674533151057096}
{'weights': [0.9523664604616587, 0.6560461028688421, 0.6856574048542334, 0.1763188217529577, 0.03986161844741546, 0.5384035539056571], 'output': 0.9154868015676352, 'delta': 0.07083187090331}
{'weights': [0.5819600247508322, 0.32773759384839196, 0.49033338141950983, 0.012959773536686113, 0.233598826370098, 0.19839537502481608], 'output': 0.8270082468803139, 'delta': -0.024749170074767995}
{'weights': [0.1131405871934549, 0.09313198884727894, 0.7438902265664203, 0.9174979321857786, 0.5989021961648644, 0.7864589614330272], 'output': 0.9477114433085216,